In [ ]:
# !pip install --upgrade tensorflow keras

In [ ]:
# from sklearn import __version__ as sklearn_version


# from sklearn.utils import all_estimators
# from sklearn.base import ClassifierMixin
# from sklearn.exceptions import NotFittedError

# import numpy as np

# from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
# from sklearn.base import BaseEstimator, ClassifierMixin
# from sklearn.utils.multiclass import unique_labels
# from sklearn.utils.validation import check_is_fitted
# from sklearn.utils.multiclass import check_classification_targets
# from sklearn.metrics import euclidean_distances
# from sklearn.preprocessing import LabelEncoder
# from sklearn.utils.extmath import safe_sparse_dot

# from keras.wrappers.scikit_learn import BaseWrapper
# from keras.wrappers.scikit_learn import KerasClassifier


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # forcibly remounting for clarity

# dataset_path = "/content/drive/My Drive/Datasets/TESS"

In [ ]:
dataset_path = 'https://drive.google.com/drive/folders/17Oxc4Pbsg9728TbjE58F-nMXcpayOFdK'
paths = []
labels = []
for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        labels.append(label.lower())
    if len(paths) == 2800:
        break
print('Dataset is Loaded')

In [ ]:
len(paths)

In [ ]:
paths[:5]

In [ ]:
labels[:5]

In [ ]:
## Create a dataframe
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels
df.head()

In [ ]:
df['label'].value_counts()

In [ ]:
sns.countplot(data=df, x='label')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.waveshow(data, sr=sr)
    plt.show()

def spectogram(data, sr, emotion):
    x = librosa.stft(data)
    xdb = librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(11,4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()

In [ ]:
emotion = 'fear'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'angry'
path = np.array(df['speech'][df['label']==emotion])[1]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'disgust'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'neutral'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'sad'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'ps'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'happy'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

In [ ]:
extract_mfcc(df['speech'][0])

In [ ]:
X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))

In [ ]:
X_mfcc

In [ ]:
X = [x for x in X_mfcc]
X = np.array(X)
X.shape

In [ ]:
## input split
X = np.expand_dims(X, -1)
X.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])


In [ ]:
y = y.toarray()

In [ ]:
y.shape

In [ ]:
# Save the encoder object
import pickle

encoder_path = '/content/drive/MyDrive/Datasets/label_encoder.pkl'  # Define the path to save the encoder file
with open(encoder_path, 'wb') as f:
    pickle.dump(enc, f)

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Dropout
# from keras.callbacks import ModelCheckpoint

# model = Sequential([
#     LSTM(256, return_sequences=False, input_shape=(40,1)),
#     Dropout(0.2),
#     Dense(128, activation='relu'),
#     Dropout(0.2),
#     Dense(64, activation='relu'),
#     Dropout(0.2),
#     Dense(7, activation='softmax')
# ])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

# # Create a checkpoint to save the best model based on validation accuracy
# checkpoint = ModelCheckpoint('/content/drive/MyDrive/Datasets/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# # Train the model with checkpoint callback
# history = model.fit(X, y, validation_split=0.2, epochs=70, batch_size=64, callbacks=[checkpoint])

In [ ]:
import numpy as np
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint

# Define a function to create the model
def create_model(optimizer='adam', dropout_rate=0.2):
    model = Sequential([
        LSTM(256, return_sequences=False, input_shape=(40,1)),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(7, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper for use in GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [50, 100, 150],
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
try:
    grid_result = grid_search.fit(X, y)
except Exception as e:
    print("Error occurred during grid search fitting:", e)


# Print the best parameters and accuracy
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Save the best model
best_model = grid_result.best_estimator_.model
best_model.save('/content/drive/MyDrive/Datasets/best_model.h5')

# Train the best model
history = best_model.fit(X, y, validation_split=0.2, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'])

# Print information about the saved model
print("Best model training completed.")
print("Accuracy:", history.history['accuracy'])
print("Validation Accuracy:", history.history['val_accuracy'])
print("Loss:", history.history['loss'])
print("Validation Loss:", history.history['val_loss'])
print("Label encoder saved as label_encoder.pkl.")


In [ ]:
import numpy as np
import librosa
from keras.models import load_model
import pickle

# Load the saved model
saved_model_path = '/content/drive/MyDrive/Datasets/best_model.h5'
loaded_model = load_model(saved_model_path)

# Load the encoder
encoder_path = '/content/drive/MyDrive/Datasets/label_encoder.pkl'
with open(encoder_path, 'rb') as f:
    label_encoder = pickle.load(f)

# Define a function to extract MFCC features from an audio file
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

# Define a function to predict emotion from an audio file
def predict_emotion(audio_file):
    # Extract MFCC features from the audio file
    mfcc_features = extract_mfcc(audio_file)
    # Reshape the features for model input
    mfcc_features = np.expand_dims(mfcc_features, axis=0)
    mfcc_features = np.expand_dims(mfcc_features, axis=-1)
    # Predict the emotion using the loaded model
    predicted_probabilities = loaded_model.predict(mfcc_features)
    # Get the predicted emotion label index
    predicted_emotion_index = np.argmax(predicted_probabilities)
    # Decode the predicted emotion label
    predicted_emotion_label = label_encoder.inverse_transform([[predicted_emotion_index]])[0][0]
    return predicted_emotion_label

# Define the path to the audio file you want to test
audio_file_path = '/content/drive/MyDrive/Datasets/TESS Toronto emotional speech set data/YAF_sad/YAF_youth_sad.wav'

# Predict the emotion from the audio file
predicted_emotion = predict_emotion(audio_file_path)

print("Predicted Emotion:", predicted_emotion)


In [ ]:
# !pip install tensorflow

In [ ]:
# from tensorflow import keras

# # Load your saved model
# model = keras.models.load_model("/content/drive/MyDrive/Datasets/best_model.h5")

# # Print model summary
# print("Model Summary:")
# model.summary()

# # Print individual layer details
# print("\nLayer Details:")
# for i, layer in enumerate(model.layers):
#     print(f"Layer {i+1}:")
#     print(f"- Name: {layer.name}")
#     print(f"- Type: {layer.__class__.__name__}")
#     print(f"- Output shape: {layer.output_shape}")
#     print(f"- Input shape: {layer.input_shape}\n")
# # Access the last layer (output layer) using indexing
# output_layer = model.layers[-1]

# # Check for expected layer structure and activation
# if output_layer.name == "dense_8" and output_layer.activation == "softmax":
#     # Assuming 7 output units corresponding to 7 emotions
#     labels = ["anger", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

#     # Check if output units have meaningful names (optional)
#     try:
#         output_names = [model.output_layer.get_weights()[1][i] for i in range(len(labels))]
#         print("\nPotential Label Mapping (if output units have names):")
#         for i, name in enumerate(output_names):
#             if name:
#                 print(f"- Unit {i+1}: {name}")
#             else:
#                 print(f"- Unit {i+1}: No name assigned")
#     except Exception as e:
#         print(f"Error checking output unit names: {e}")
# else:
#     print("Output layer configuration might not align with label mapping assumptions. Adjust the code based on your model's structure.")


In [ ]:
# # Define function to extract MFCC features
# def extract_mfcc(filename):
#   y, sr = librosa.load(filename, duration=3, offset=0.5)
#   mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
#   return mfcc

In [ ]:
# import tensorflow as tf
# # Load the saved model
# model = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/best_model.h5')  # Replace with your actual path


In [ ]:
# # Function for prediction
# def predict_emotion(audio_file):
#   # Extract MFCC features
#   mfcc_features = extract_mfcc(audio_file)
#   mfcc_features = np.expand_dims(mfcc_features, axis=0)

#   # Make prediction
#   prediction = model.predict(mfcc_features)
#   predicted_emotion = np.argmax(prediction)

#   # Map prediction to emotion label
#   # Assuming you have a label mapping defined elsewhere
#   emotion_labels = ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
#   predicted_emotion = emotion_labels[predicted_emotion]

#   return predicted_emotion

In [ ]:
# # Example usage
# audio_file_path = '/content/drive/MyDrive/Datasets/TESS Toronto emotional speech set data/YAF_sad/YAF_youth_sad.wav'  # Replace with your actual path
# predicted_emotion = predict_emotion(audio_file_path)

# print(f"Predicted emotion: {predicted_emotion}")